## Go through IWOAI data and correct any errors

### I know that there is at least one case of masks in wrong order

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [6]:
# work out where we are by printing contents of a given path
IWOAI_path = '../data/IWOAI'
print(os.listdir(IWOAI_path))

['valid_copy.tar.gz', 'test', 'test_copy.tar.gz', 'valid', 'train', 'train_copy.tar.gz', 'test_gt_copy.zip']


In [9]:
# get list of paths for all images/masks
train_images = glob.glob(IWOAI_path + '/train/*.im')
valid_images = glob.glob(IWOAI_path + '/valid/*.im')
test_images = glob.glob(IWOAI_path + '/test/*.im')

In [10]:
# print counts
print('Number of training images:', len(train_images))
print('Number of validation images:', len(valid_images))
print('Number of test images:', len(test_images))

Number of training images: 120
Number of validation images: 28
Number of test images: 28
